<a href="https://colab.research.google.com/github/ajit-gvs/HCR/blob/master/main_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

import cv2
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from PIL import Image
import math
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
directory='/content/data'
text=open('/content/words_new.txt')






TensorFlow 1.x selected.
1.15.2


In [ ]:
def image_binarization(image):
  # converting image to grayscale
  image_grayscale=image.convert('L')
  img=np.array(image_grayscale) 
   
  img[img <128] = 0
  img[img >=128] = 254 

  #making white as foreground pixels and black as background pixels
  img[img==0]=255
  img[img==254]=0
  
  return img


def line_segmentation(img):
  start_matrix=[]
  end_matrix=[]

  #matrix to get the start and end points of a line
  
  lines=[]
  begin_matrix=[]
  stop_matrix=[]
  del_start_matrix=[]
  del_end_matrix=[]

  horizontal_hist = np.sum(img,axis=1,keepdims=True)/255
  start_count=0

  for i in range(len(horizontal_hist)):

    if horizontal_hist[i]>0 and horizontal_hist[i-1]==0:
      start_count+=1
      start_matrix.append(i)

    if horizontal_hist[i]==0 and start_count>0 and horizontal_hist[i-1]>0:
      end_matrix.append(i)

  
  
  if len(start_matrix)==len(end_matrix):
    for i in range(len(start_matrix)):
      if end_matrix[i]-start_matrix[i]<20:
        del_start_matrix.append(i)
        del_end_matrix.append(i)
    for i in range(len(start_matrix)):
      count=0
      for j in range(len(del_start_matrix)):
        if i==del_start_matrix[j]:
          count=count+1
      if count==0:
        begin_matrix.append(start_matrix[i])
    for i in range(len(end_matrix)):
      count=0
      for j in range(len(del_end_matrix)):
        if i==del_end_matrix[j]:
          count=count+1
      if count==0:
        stop_matrix.append(end_matrix[i])
    
    for i in range(len(begin_matrix)):
      lines.append(img[begin_matrix[i]:stop_matrix[i],:])

      

      

  
  return lines




def word_segmentation(img):
  start_matrix=[]
  end_matrix=[]

  #matrix to get the start and end points of a word
  dissection_matrix=[]
  
  words=[]
  m,n=img.shape
  
  length=[]
  vertical_hist = np.sum(img,axis=0,keepdims=True)/255
  
  
  start_count=0
  
  
  for i in range(len(vertical_hist[0])):
    if vertical_hist[0][i]>0 and vertical_hist[0][i-1]==0:
      start_count+=1
      start_matrix.append(i)
    if vertical_hist[0][i]==0 and start_count>0 and vertical_hist[0][i-1]>0:
      end_matrix.append(i)
  
  
  length_mag=0
  for i in range(len(start_matrix)):
    if i>0:
      length_mag=(start_matrix[i]-end_matrix[i-1])
      length.append(length_mag)

  max=np.max(length)
     
  avg=max/3

  dissection_matrix.append([start_matrix[0],end_matrix[0]])
  j=0
  for i in range(len(length)-1):
    
    if length[i]> avg:
      dissection_matrix.append([start_matrix[i+1],end_matrix[i+1]])
      j=j+1
      
    if length[i]<=avg:
      dissection_matrix[j][1]=end_matrix[i+1]
  for i in range(len(dissection_matrix)):
    words.append(img[0:m,dissection_matrix[i][0]:dissection_matrix[i][1]])
  
  return words


def char_segmentation(img):
  start_matrix=[]
  dissection_matrix=[]
  delete_matrix=[]
  address_matrix=[]
  
  
  characters=[]
  m,n=img.shape

  vertical_hist = np.sum(img,axis=0,keepdims=True)/255
  start_matrix.append(0)

  for i in range(len(vertical_hist[0])):
    if vertical_hist[0][i]<10:
      start_matrix.append(i)

  for i in range(len(start_matrix)-1):
    if start_matrix[i+1]-start_matrix[i]<10:
      delete_matrix.append(i)
  
  for i in range(len(start_matrix)):
    count=0
    for j in range(len(delete_matrix)):
      if  i==delete_matrix[j]:
        count=count+1
    if count==0:
      address_matrix.append(start_matrix[i])


  for i in range(len(address_matrix)-1):
    dissection_matrix.append([address_matrix[i],address_matrix[i+1]])
  for i in range(len(dissection_matrix)):
    characters.append(img[0:m,dissection_matrix[i][0]:dissection_matrix[i][1]])
    
    
  return characters

In [ ]:
def data_label():

  samples=[]
  labels=[]

  count=0
  
 
  text=open('/content/words_new.txt')
  
  for filename in os.scandir(directory):
    image_orig=Image.open('/content/data/'+filename,'r')
    img=image_binarization(image_orig)
    characters=char_segmentation(img)
    for line in text:
      line_split=line.strip().split(' ')
      filename_split=filename.strip().split('.')
      if filename_split[0] == line_split[0]:
        if len(characters) == len(line_split[8]):
          for i in range(len(characters)):
            character=cv2.resize(characters[i],(28,28))
            sample.append(character)
          for i in line_split[8]:
            labels.append(i) 
   
          
    
  
                 
  
  return  samples,labels

samples,labels=data_label()
print(len(samples))
print(len(labels))


NameError: ignored

In [ ]:
samples = np.array(samples, dtype='float' )/ 255.0
labels = np.array(labels)
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels
samples.shape

(203, 28, 28)

In [ ]:
from sklearn.model_selection import train_test_split
trainx,testx,trainy,testy = train_test_split(samples, labels, test_size=0.2, random_state=42)


In [ ]:
len(testx)
testx.shape
trainx=trainx.reshape(trainx.shape[0],28,28,1)
testx=testx.reshape(testx.shape[0],28,28,1)


In [ ]:
HP_epoch = 100
HP_init_lr = 1e-3 # learning_rate = 0.001
HP_batch_size = 32
HP_image_dims = (28,28,1)
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
HP_model_path = 'bin/model'
HP_binarized_labels = 'bin/labels'
HP_metrics_storage = 'eval'

In [ ]:
aug = ImageDataGenerator( rotation_range=25, width_shift_range=0.1, 
                         height_shift_range=0.1, shear_range=0.2,
                         zoom_range=0.2, horizontal_flip=True,fill_mode='nearest')

In [ ]:
from keras import backend
from keras.layers.core import Dense, Dropout, Flatten, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
#N= 5
HP_block1_conv_dim = 32
HP_block2_conv_dim = 64
HP_block3_conv_dim = 128
HP_block4_conv_dim = 256
HP_block5_dense_dim = 1024
HP_small_pattern = (3,3) # UNCOMPRESSED or 1-2 compression IMAGES
HP_large_pattern = (2,2) # 4 times compressed images from previous MP layers!!!
HP_dropout_little =0.25
HP_dropout_big = 0.50
# HP_epochs, batch_size-> are now problems of the developer USING this model. 

HP_img_dims = (28,28,1)

In [ ]:
class RacoonVGG:
  @staticmethod
  def build(height, width,depth, classes):
    input_shape = (height, width, depth)
    channel_dim = -1 # last position 
    if backend.image_data_format() == 'channels_first':
      input_shape = (depth, height, width)
      channel_dim = 1
    model = Sequential()
    # BLOCK1
    model.add(Conv2D(HP_block1_conv_dim,HP_small_pattern, padding='same',
                     input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(MaxPooling2D(pool_size=HP_small_pattern))
    model.add(Dropout(HP_dropout_little))

    # COMPLEX BLOCK 2
    model.add(Conv2D(HP_block2_conv_dim,HP_small_pattern, padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(Conv2D(HP_block2_conv_dim,HP_small_pattern, padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(MaxPooling2D(pool_size=HP_large_pattern))
    model.add(Dropout(HP_dropout_little))
    
    # COMPLEX BLOCK 3
    model.add(Conv2D(HP_block3_conv_dim,HP_small_pattern, padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(Conv2D(HP_block3_conv_dim,HP_small_pattern, padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(MaxPooling2D(pool_size=HP_large_pattern))
    model.add(Dropout(HP_dropout_little))

    # COMPLEX BLOCK 4
    model.add(Conv2D(HP_block4_conv_dim,HP_small_pattern, padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(Conv2D(HP_block4_conv_dim,HP_small_pattern, padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(MaxPooling2D(pool_size=HP_large_pattern))
    model.add(Dropout(HP_dropout_little))

    # BLOCK 5- Image Classification (OBJECT)
    model.add(Flatten())
    model.add(Dense(HP_block5_dense_dim))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(HP_dropout_big))
    model.add(Dense(classes))
    model.add(Activation('softmax'))
    return model


trainx=trainx.reshape(trainx.shape[0],28,28,1)
testx=testx.reshape(testx.shape[0],28,28,1)


In [ ]:
import keras
import keras.utils
from keras import utils as np_utils
from keras.optimizers import adam
model = RacoonVGG.build(height=HP_img_dims[0], width=HP_img_dims[1],depth=HP_img_dims[2],classes = len(lb.classes_))
opt = keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
hist = model.fit(aug.flow(trainx, trainy, batch_size=HP_batch_size),
                 validation_data=(testx, testy),
                 steps_per_epoch= len(trainx) // HP_batch_size,
                 epochs=HP_epoch, verbose=1)

Epoch 1/100
5/5 [==============================] - 3s 612ms/step - loss: 4.6370 - accuracy: 0.0312 - val_loss: 3.5335 - val_accuracy: 0.0732
Epoch 2/100
5/5 [==============================] - 1s 154ms/step - loss: 4.7499 - accuracy: 0.0300 - val_loss: 3.5085 - val_accuracy: 0.1220
Epoch 3/100
5/5 [==============================] - 1s 186ms/step - loss: 5.0422 - accuracy: 0.0462 - val_loss: 3.4866 - val_accuracy: 0.1220
Epoch 4/100
5/5 [==============================] - 1s 225ms/step - loss: 5.0700 - accuracy: 0.0437 - val_loss: 3.4653 - val_accuracy: 0.1220
Epoch 5/100
5/5 [==============================] - 1s 174ms/step - loss: 4.5241 - accuracy: 0.0200 - val_loss: 3.4487 - val_accuracy: 0.1220
Epoch 6/100
5/5 [==============================] - 1s 214ms/step - loss: 5.0481 - accuracy: 0.0500 - val_loss: 3.4372 - val_accuracy: 0.1220
Epoch 7/100
5/5 [==============================] - 1s 167ms/step - loss: 4.9781 - accuracy: 0.0462 - val_loss: 3.4256 - val_accuracy: 0.0488
Epoch 8/100
5

In [ ]:
def data_label():

  samples=[]
  labels=[]

  count=0
  
 
  text=open('/content/words_new.txt')
  
  for filename in os.listdir(directory):
    image_orig=Image.open('/content/data/'+filename,'r')
    img=image_binarization(image_orig)
    characters=char_segmentation(img)
    for line in text:
      line_split=line.strip().split(' ')
      filename_split=filename.strip().split('.')
      if filename_split[0] == line_split[0]:
        if len(characters) == len(line_split[8]):
          for i in range(len(characters)):
            character=cv2.resize(characters[i],(28,28))
            sample.append(character)
          for i in line_split[8]:
            labels.append(i) 
   
          
    
  
                 
  
  return  samples,labels

samples,labels=data_label()
print(len(samples))
print(len(labels))

FileNotFoundError: ignored